In [12]:
import os

In [13]:
%pwd

'd:\\Personal\\Projects_Computer_Science_And_Engineering\\RaunakDLProject\\End-to-End-Chest-Cancer-Classification-using-MLflow-DVC'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\Personal\\Projects_Computer_Science_And_Engineering\\RaunakDLProject\\End-to-End-Chest-Cancer-Classification-using-MLflow-DVC'

In [22]:
import tensorflow as tf
print("Is eager execution enabled? ", tf.executing_eagerly())


Is eager execution enabled?  True


In [31]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [32]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [33]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [34]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [35]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    # def get_base_model(self):
    #     self.model = tf.keras.models.load_model(
    #         self.config.updated_base_model_path
    #     )

    def get_base_model(self):
    # Load the base model
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

        # Recompile the model with the new optimizer
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
    )


    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )




    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
    

In [36]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-01-06 13:54:21,614: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-06 13:54:21,773: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-06 13:54:21,851: INFO: common: created directory at: artifacts]
[2025-01-06 13:54:21,908: INFO: common: created directory at: artifacts\training]
[2025-01-06 13:54:39,879: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
17/17 ━━━━━━━━━━━━━━━━━━━━ 271s 13s/step - accuracy: 0.6085 - loss: 1.0062 - val_accuracy: 1.0000 - val_loss: 0.0534
[2025-01-06 13:59:23,754: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`

In [12]:
import tensorflow as tf
print(tf.__version__)


2.18.0


In [26]:
import tensorflow as tf
import numpy as np
print("TensorFlow version:", tf.__version__)
print("NumPy version:", np.__version__)


TensorFlow version: 2.18.0
NumPy version: 2.0.2


In [20]:
import tensorflow as tf
import numpy as np
print("TensorFlow version:", tf.__version__)
print("NumPy version:", np.__version__)


TensorFlow version: 2.16.1
NumPy version: 1.26.4


In [21]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

# Training process
training.train()


d:\Personal\Projects_Computer_Science_And_Engineering\RaunakDLProject\dlproj\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


ValueError: Unknown variable: <Variable path=dense/kernel, shape=(25088, 2), dtype=float32, value=[[ 0.00342126  0.00741317]
 [ 0.00544174  0.00753661]
 [-0.01456405  0.014938  ]
 ...
 [ 0.0041233  -0.0108095 ]
 [ 0.00448383  0.00605257]
 [ 0.00184711  0.00351097]]>. This optimizer can only be called for the variables it was originally built with. When working with a new set of variables, you should recreate a new optimizer instance.